
### Done
* Forward pass


### Missing
* preprocessing
* Backwards pass
* Summaries
* Replay buffer
* target network
* Save model
* Breakout 
* Let it train

# DQN on Breakout



In [ ]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run all cells',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.execute_all_cells();
        return false;
    }
});

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline

In [ ]:
import numpy as np
import tensorflow as tf
import gym
import keras 

import utils

In [ ]:
## Hyperparameters

batch_size = 32
training_time = int(50e6) # frames
training_time = int(1e5) # frames

replay_buffer_size = int(1e6)
replay_buffer_size = int(1e4)

learning_rate = 0.00025
gamma = 0.99
epsilon_init = 1.0
epsilon_end = 0.1
annealing_period = 1e6 # frames


In [ ]:
## Parameters
env_name = 'Breakout-v0'
env_name = 'CartPole-v0'



In [ ]:
## Derived settings
annealer = utils.Annealer(epsilon_init, epsilon_end, annealing_period)

env = gym.make(env_name)
obs = env.reset()
s_size = list(obs.shape)
a_size = env.action_space.n

obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+s_size, name='obsPlaceholder')
# obsPlaceholder = tf.placeholder(tf.float32, shape=[None]+[84, 84, 4], name='obsPlaceholder')
# Assume action is encoded as ONE number
actionPlaceholder = tf.placeholder(tf.int32, shape=[None], name='actionPlaceholder')
actions_onehot = tf.one_hot(actionPlaceholder, a_size, dtype=tf.float32)
rewardPlaceholder = tf.placeholder(tf.float32, shape=[None], name='rewardPlaceholder')
targetQPlaceholder = tf.placeholder(tf.float32, shape=[None], name='targetQPlaceholder')

print('-- Placeholders --')
print('obsPlaceholder', obsPlaceholder.get_shape())
print('actionPlaceholder', actionPlaceholder.get_shape())
print('actions_onehot', actions_onehot.get_shape())
print('rewardPlaceholder', rewardPlaceholder.get_shape())
print('targetQPlaceholder', targetQPlaceholder.get_shape())
print()

### Create model
from models import DQN
# model = DQN(obsPlaceholder, a_size)
from models import simple_dense
model = simple_dense(obsPlaceholder, a_size, [64, 64])

# TODO: Encapsulate all this crap!
qvalues = model.output 
qvalue = tf.reduce_sum(tf.multiply(qvalues, actions_onehot), axis=1)
qmax = tf.reduce_max(qvalues, axis=1)
action = tf.argmax(qvalues, axis=1) 

print("-- Internal tensors --")
print('qvalues', qvalues.get_shape())
print('qvalue', qvalue.get_shape())
print('action', action.get_shape())
print('qmax', qmax.get_shape())

loss = tf.reduce_mean(tf.square(targetQPlaceholder - qvalue)) 
train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(loss) 



In [ ]:
# utils.Experience_buffer().test()

In [ ]:
update_frequency = 32 
assert update_frequency > 1, "The dimensions doen't allign if update_frequency" \
    + 'is less than 1: ' + str(update_frequency)
minimum_experience = 32


buffer = utils.Experience_buffer(replay_buffer_size)

try:
    sess.close()
except:
    pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

obs = env.reset()
for f in range(training_time):
    epsilon = annealer.linear(f)
#     action_chosen = sess.run(action, feed_dict={obsPlaceholder : [obs]}) if np.random.rand(1) > epsilon \
#         else np.random.randint(a_size)
    [action_chosen] = sess.run(action, feed_dict={obsPlaceholder : [obs]})
    assert 'int' in str(type(action_chosen))
    next_obs, reward, done, _ = env.step(action_chosen)
    # TODO: Preprocess obs
    # TODO: wrap in action repeater

    experience = {'obs':[obs], 'action':action_chosen, 'reward':reward,
                  'next_obs':[next_obs], 'done':done}
    buffer.add(experience)
    obs = next_obs

    if f % update_frequency == 0 \
        and f > minimum_experience:
        train_batch = buffer.sample(batch_size)

        # Compute ... TODO: add description
        Q1 = sess.run(qmax, feed_dict={
                obsPlaceholder : train_batch['next_obs']
            })
        target = train_batch['reward'] + gamma*Q1
        # TODO: where train_batch['done']: change to r

#         print("train_batch['obs']")
#         print(type(train_batch['obs']))
#         print(train_batch['obs'].shape)
#         print()

#         print('Q1')
#         print(type(Q1))
#         print(Q1.shape)
#         print()

#         print("train_batch['action']")
#         print(type(train_batch['action']))
#         print(train_batch['action'].shape)
#         print()

#         print("target")
#         print(type(target))
#         print(target.shape)
#         print()

        Q1_dict = {obsPlaceholder : train_batch['obs'],
                   actionPlaceholder : train_batch['action'],
                   targetQPlaceholder : target # TODO: Should be target network!
            }
        _ = sess.run(train_op, feed_dict=Q1_dict)
    if f % 1000 == 0:
        print('{:9}'.format(f), end=' ')
    if done:
        obs = env.reset()
        

# env.render(close=True)

In [ ]:
## Run and render a forward pass

obs = env.reset()
for f in range(training_time):
    [action_chosen] = sess.run(action, feed_dict={obsPlaceholder : [obs]})
    assert 'int' in str(type(action_chosen))
    obs, reward, done, _ = env.step(action_chosen)
    env.render()

    if done:
        break
        


In [ ]:
env.render(close=True)